In [379]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 500)
from tqdm import tqdm_notebook
import seaborn as sns

import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer, PorterStemmer 

from sklearn.metrics.pairwise import euclidean_distances, manhattan_distances, cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from itertools import islice
from scipy.spatial.distance import cdist
from scipy.stats.stats import pearsonr

In [91]:
import googletrans
from googletrans import Translator
import time
import re

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Individual request

In [ ]:
from requests import get
url = 'https://birrapedia.com/cervezas/de-espana'
response = get(url)
print(response.text[:500])

In [ ]:
response

In [ ]:
response.text

# Beatiful Soup

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
print(soup.prettify())

In [ ]:
set([type(item) for item in list(soup.children)])

In [ ]:
soup.find_all('a')

In [ ]:
soup.find_all('p')[0].get_text()

In [ ]:
soup.find_all('p')[4].get_text()

In [ ]:
soup.select("div p")

In [ ]:
for i in range(10):
    print(soup.find_all('p')[i].get_text())

In [ ]:
divs = soup.findAll("div", {"class": "lista-cab"})

In [ ]:
divs

In [ ]:
soup.findAll("div", {"class": "lista-cab"})[0]

In [ ]:
soup.findAll("div", {"class": "lista-cab"})[1]

In [ ]:
soup.findAll("div", {"class": "lista-cab"})[1].get_text()

In [ ]:
soup.findAll("div", {"class": "lista-cab"})[3].get_text()

In [ ]:
soup.findAll("div", {"class": "lista-cab"})[3]

In [ ]:
soup.findAll("div", {"class": "lista-cab"})[3].findAll("p", {"class": "colorNegro"})[0].get_text()

In [ ]:
soup.findAll("div", {"class": "lista-cab"})[3].findAll("p", {"class": "colorNegro"})[1].get_text()

In [ ]:
soup.findAll("div", {"class": "lista-cab"})[3].findAll("p", {"class": "text-muted oculta-pequenyo"})[0].get_text()

In [ ]:
N_beers = len(soup.findAll("div", {"class": "lista-cab"}))

In [ ]:
beer_name = []
beer_style = []
description = []

for i in range(N_beers):
    beer_name.append(soup.findAll("div", {"class": "lista-cab"})[i].findAll("p", {"class": "colorNegro"})[0].get_text())
    beer_style.append(soup.findAll("div", {"class": "lista-cab"})[i].findAll("p", {"class": "colorNegro"})[1].get_text())
    description.append(soup.findAll("div", {"class": "lista-cab"})[i].findAll("p", {"class": "text-muted oculta-pequenyo"})[0].get_text())

# Automatization

In [ ]:
url_1 = 'https://birrapedia.com/espana/'
url_2 = '/i-70616573'

In [ ]:
beer_name = []
beer_style = []
description = []

for i in tqdm_notebook(range(200)):
    print('Iteration', i)
    # Build url
    j = i + 1
    url = url_1 + str(j) + url_2
    
    # Make request
    response = get(url)

    # Parse content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')    
    
    N_beers = len(soup.findAll("div", {"class": "lista-cab"}))  

    for i in range(N_beers):
        beer_name.append(soup.findAll("div", {"class": "lista-cab"})[i].findAll("p", {"class": "colorNegro"})[0].get_text())
        beer_style.append(soup.findAll("div", {"class": "lista-cab"})[i].findAll("p", {"class": "colorNegro"})[1].get_text())
        description.append(soup.findAll("div", {"class": "lista-cab"})[i].findAll("p", {"class": "text-muted oculta-pequenyo"})[0].get_text())

In [ ]:
len(beer_name)

In [ ]:
df = pd.DataFrame(list(zip(beer_name, beer_style, description)), columns =['Name', 'Style', 'Description']) 

# Export raw to csv

In [ ]:
df.to_csv('Beer_dataset_raw.csv')

# Cleaning

In [6]:
df = pd.read_csv('Beer_dataset_raw.csv', index_col=0)

In [7]:
df.isna().sum()

Name            0
Style           0
Description    66
dtype: int64

In [ ]:
df.head()

## Style

In [ ]:
df[['Style','ABV', 'IBU']] = df.Style.str.split(' - ', n=2, expand=True)

In [ ]:
df.head()

## ABV

In [ ]:
df.ABV.head()

### Replace strange symbols

In [ ]:
df.ABV = df.ABV.str.replace(" º", "")
df.ABV = df.ABV.str.replace(" ", "")
df.ABV = df.ABV.str.replace("\xa0", " ")
df.ABV = df.ABV.str.replace("%", "")
df.ABV = df.ABV.str.replace("vol.", "")
df.ABV = df.ABV.str.replace("ABV", "")

In [ ]:
# Eliminamos "Hidromiel" y "Sidra"

df = df[df.ABV != "Applewine"]
df = df[df.ABV != "Hidromiel"]
df = df[df.ABV != "OtherSpecialtyCider/Perry"]
df = df[df.ABV != "FruitCider"]
df = df[df.ABV != "CTZ.Amarillo.Mosaic"]

In [ ]:
# Eliminamos los 13 registros que no traen valor de ABV, solo de IBU

df = df[~df.ABV.str.contains("IBU", na=False)]

In [ ]:
df.ABV = df.ABV.str.split("-", n=1, expand=True)[0]

In [ ]:
df.ABV = df.ABV.str.replace("\.\.", ".", regex=True)
df.ABV = df.ABV.str.replace("’", "")
df.ABV = df.ABV.str.replace("´", "")

In [ ]:
df.ABV = df.ABV.str.split("/", n=1, expand=True)[0]

In [ ]:
df.ABV.isna().sum()

In [ ]:
df.ABV = df.ABV.astype('float')

### Clean outliers

In [ ]:
df.ABV = df.ABV.apply(lambda x: x if x<30 else x/10)

## IBU

In [ ]:
df.IBU = df.IBU.str.replace("IBU", "")
df.IBU = df.IBU.replace(" ", "")
df.IBU = df.IBU.str.replace("\xa0", " ")
df.IBU = df.IBU.str.replace(",", ".")

In [ ]:
df.IBU = df.IBU.str.split("-", n=1, expand=True)[0]
df.IBU = df.IBU.str.split("/", n=1, expand=True)[0]

In [ ]:
df.IBU = df.IBU.astype('float')

## Name, Brewery and Rating

In [ ]:
df.Name = df.Name.str.replace("_borderstar", "")
df.Name = df.Name.str.replace("star", "")
df.Name = df.Name.str.replace("_half", "")

In [ ]:
df[['Name', 'Rating']] = df.Name.str.split('_border', n=1, expand=True)

In [ ]:
df.Rating = df.Rating.str.replace(",", ".")

In [ ]:
df.Rating = df.Rating.astype('float')

In [ ]:
df.head()

In [ ]:
df["Test"] = df.Name.str.split('/')

In [ ]:
df["Name"] = df.Test.apply(lambda x: x[1] if len(x)>1 else x[0])
df["Brewery"] = df.Test.apply(lambda x: x[0] if len(x)>1 else None)

In [ ]:
df.Name = df.Name.str.rstrip()
df.Name = df.Name.str.lstrip()

df.Brewery = df.Brewery.str.rstrip()
df.Brewery = df.Brewery.str.lstrip()

## Reordering

In [ ]:
columns = ['Name', 'Brewery', 'Style', 'ABV', 'IBU', 'Rating', 'Description']

df = df[columns]

In [ ]:
df.head()

In [ ]:
df.Brewery.notnull().sum()

## Duplicates

In [ ]:
df.Name.value_counts().head(10)

In [ ]:
df.Brewery.value_counts().head(10)

In [ ]:
df[df.Name == "La Quince"]

TO DO:
    - Corregir valores erróneos de Name y Brewery

In [ ]:
# Drop some duplicates after checking which one we want

# df = df.drop(4359)
# df = df.drop(679)
# df = df.drop(4464)
# df = df.drop(3831)

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df.head()

In [ ]:
df.to_csv('Beer_dataset_v1.csv')

# Read csv

In [440]:
df = pd.read_csv('Beer_dataset_v1.csv', index_col=0, )

In [441]:
df.Description = df.Description.fillna('')

# Merge to fill description

In [442]:
descr_df = pd.read_csv('dataset_ramon.csv')

In [443]:
descr_df = descr_df[['Name', 'Description']]
descr_df.rename(columns={'Description': 'Description_full'}, inplace=True)

In [444]:
df.Name.nunique()

7846

In [445]:
descr_df.Name.nunique()

7881

In [446]:
df = df.merge(descr_df, on='Name', how='inner')

In [447]:
df.Description_full = df.Description_full.fillna('')

# EDA

In [ ]:
sns.distplot(df[df.ABV.notnull()]['ABV'])

In [ ]:
sns.distplot(df[df.IBU.notnull()]['IBU'])

In [ ]:
sns.distplot(df[df.Rating.notnull()]['Rating'])

# Groupby

In [ ]:
df_grouped = df.groupby('Style').agg({'ABV': ['mean', 'count'],
                                      'IBU': ['mean', 'count'],
                                      'Rating': ['mean', 'count']}).reset_index()

In [ ]:
df_grouped.columns = df_grouped.columns.droplevel(0)
df_grouped.columns = ['Style', 'ABV_mean', 'ABV_count', 'IBU_mean', 'IBU_count', 'Rating_mean', 'Rating_count']

In [ ]:
df_grouped.sort_values(by='IBU_mean', ascending=False).head()

In [ ]:
df[df.Style == "Flanders Brown Ale-Oud Bruin"]

# NLP

## Stopwords

In [448]:
stopwords_list = stopwords.words('english') + stopwords.words('spanish')

In [449]:
column = 'Description_full'

df['Description_2'] = df[column].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords_list)]))
df["Description_2"] = df['Description_2'].map(lambda x: re.sub(r'\W+', ' ', x))
df["Description_2"] = df['Description_2'].str.lower()

In [450]:
df[['Description', 'Description_2']].sample(5)

,Description,Description_2
4630,Áncora O Afiador\r\nCerveza clásica de trigo con aromas a lúpulos europeos y una espuma blanca y persistente.,áncora o afiador cerveza clásica trigo aromas lúpulos europeos espuma blanca persistente
6284,La Sin de Tropical.,la sin tropical
1113,"Palisandro\nBrown Ale\nMarrón rojiza. Aromas torrefactados y herbáceos. Corpulenta, carbonatación media con notas a caramelo, fruta madura con equilibrado amargor final. Icono REFUgiano.",palisandro brown ale marrón rojiza aromas torrefactados herbáceos corpulenta carbonatación media notas caramelo fruta madura equilibrado amargor final icono refugiano
2360,"Desiderata Doble Malta\r\nCerveza de color cobrizo y con buen cuerpo. Presenta un grado alcohólico medio-alto y mucho carácter debido a su especial selección de maltas tostadas.\r\nMarida con entrantes, carnes y estofados",desiderata doble malta cerveza color cobrizo buen cuerpo presenta grado alcohólico medio alto carácter debido especial selección maltas tostadas marida entrantes carnes estofados
386,Loading...\nDIPA,loading dipa


In [451]:
df.head()

,Name,Brewery,Style,ABV,IBU,Rating,Description,Description_full,Description_2
0,BlackLab Sour Lady,NaN,Berliner Weisse,5.0,NaN,NaN,Sour Lady\nBerliner Weisse with fruit\nUna cerveza ácida con frambuesas.\n---------------\nSour beer with raspberries,Sour Lady\nBerliner Weisse with fruit\nUna cerveza ácida con frambuesas.\n---------------\nSour beer with raspberries,sour lady berliner weisse fruit una cerveza ácida frambuesas sour beer raspberries
1,Blacklab Claudia Passion,NaN,American IPA,NaN,NaN,NaN,Claudia Passion\nFruit IPA\nNuestra mejor cerveza con Maracuyá\n--------------\nOur best IPA with passion fruit,Claudia Passion\nFruit IPA\nNuestra mejor cerveza con Maracuyá\n--------------\nOur best IPA with passion fruit,claudia passion fruit ipa nuestra mejor cerveza maracuyá our best ipa passion fruit
2,BlackLab More Sparkle,NaN,Brut IPA,7.0,NaN,NaN,More Sparkle\nBrut IPA\nDry and aromatic. Low bitterness.\n-----------\nMuy seca y aromática. Poco amargor.,More Sparkle\nBrut IPA\nDry and aromatic. Low bitterness.\n-----------\nMuy seca y aromática. Poco amargor.,more sparkle brut ipa dry aromatic low bitterness muy seca aromática poco amargor
3,Greyhound Brewers Give ’em Hops,NaN,Imperial IPA,8.0,NaN,NaN,Give ’em Hops\nDoble NEIPA,Give ’em Hops\nDoble NEIPA,give em hops doble neipa
4,Greyhound Brewers Slave of New Hops,NaN,American IPA,6.7,50.0,3.76,Slave of New Hops\nIPA,Slave of New Hops\nIPA,slave new hops ipa


## Stemming

In [452]:
# Ejecutar esto cuando tengamos las traducciones realizadas

# stemmer = SnowballStemmer("spanish")

# df['Stemmed'] = df["Description_2"].map(lambda x: ' '.join([stemmer.stem(y) for y in x.split(' ')]))
# df["stemmed_clean"] = df['stemmed'].map(lambda x: re.sub(r'\W+', ' ', x))

## Unique words

In [453]:
df["Description_3"] = df["Description_2"].apply(lambda x: pd.unique(x.split(' ')))
df["Description_3"] = df["Description_3"].apply(lambda x: ' '.join(x))

## PorterStemmer

In [303]:
# TO DO: Revisar para hacer esto en castellano, quizás usando Spacy
stemmer = PorterStemmer()

df['Description_3'] = df["Description_3"].map(lambda x: ' '.join([stemmer.stem(y) for y in x.split(' ')]))

## Remove words

In [454]:
list_words_to_drop = ['cerveza', 'la', 'una', 'elaborada', 'el']

In [455]:
df["Description_3"] = df["Description_3"].apply(lambda x: [word for word in x.split() if word not in list_words_to_drop])
df["Description_3"] = df["Description_3"].apply(lambda x: ' '.join(x))

In [457]:
df[['Description_full', 'Description_2', 'Description_3']].sample(5)

,Description_full,Description_2,Description_3
3507,"Mahou Cinco Estrellas Sin Gluten nace como la cerveza apta para celiacos que mantiene intacto el sabor de su hermana Mahou Cinco Estrellas. Con un color dorado brillante, destaca por ser cremosa y consistente, con un cuerpo intenso y equilibrado capaz ... Ver más satisfacer a los paladares más exigentes.\nCon el mismo sabor que la Mahou Cinco Estrellas, la nueva Mahou Sin Gluten mantiene los rasgos y procesos de elaboración característicos de su hermana mayor que nos acompaña desde 1969.\nUt...",mahou cinco estrellas sin gluten nace cerveza apta celiacos mantiene intacto sabor hermana mahou cinco estrellas con color dorado brillante destaca ser cremosa consistente cuerpo intenso equilibrado capaz ver satisfacer paladares exigentes con mismo sabor mahou cinco estrellas nueva mahou sin gluten mantiene rasgos procesos elaboración característicos hermana mayor acompaña 1969 utilizando mejores variedades lúpulo levadura obtiene cerveza dorada cremosa aroma afrutado apta celiacos consígue...,mahou cinco estrellas sin gluten nace apta celiacos mantiene intacto sabor hermana con color dorado brillante destaca ser cremosa consistente cuerpo intenso equilibrado capaz ver satisfacer paladares exigentes mismo nueva rasgos procesos elaboración característicos mayor acompaña 1969 utilizando mejores variedades lúpulo levadura obtiene dorada aroma afrutado consíguela precio punto venta habitual disfrútala bar favorito casa dejándote llevar todos los principales ingredientes son menos
4578,"BIIR Equinox Triple IPA\r\nInspirada en les Triple IPA de Califòrnia, hem creat aquesta cervesa extrema, per als cervesers més exigents, amb una gran quantitat de malts d’ordi i llupol Equinox. Malgrat la seva graduació alcohòlica i amargor es ... Ver más d’una cervesa ben equilibrada.\r\n—\r\nInspired by Californian Triple IPA, we have brewed this extreme beer for the most exigent beer lovers, whith lots of malts and Equinox hops. Despite it’a alcohol volume and bitternes, it’s a very well ...",biir equinox triple ipa inspirada triple ipa califòrnia hem creat aquesta cervesa extrema per als cervesers més exigents amb gran quantitat malts d ordi llupol equinox malgrat seva graduació alcohòlica amargor ver d una cervesa ben equilibrada inspired californian triple ipa brewed extreme beer exigent beer lovers whith lots malts equinox hops despite it a alcohol volume bitternes it s well balanced beer inspirada triple ipa california creado cerveza extrema cerveceros exigentes gran cantida...,biir equinox triple ipa inspirada califòrnia hem creat aquesta cervesa extrema per als cervesers més exigents amb gran quantitat malts d ordi llupol malgrat seva graduació alcohòlica amargor ver ben equilibrada inspired californian brewed extreme beer exigent lovers whith lots hops despite it a alcohol volume bitternes s well balanced california creado cerveceros exigentes cantidad malta cebada lúpulo pese graduación alcohólica trata bien menos
4457,Maiku 12 Mordors\r\nSummer Ale,maiku 12 mordors summer ale,maiku 12 mordors summer ale
1769,"Península Porter\r\nCerveza oscura con notas a malta tostada, aromas a chocolate y cremosa espuma que te cubre la boca.",península porter cerveza oscura notas malta tostada aromas chocolate cremosa espuma cubre boca,península porter oscura notas malta tostada aromas chocolate cremosa espuma cubre boca
499,"Don’t Hop Back In Anger\nDoble IPA\nHay canciones que se convierten en himnos y plantas que se convierten en cervezas. En ambos casos, lo importante es disfrutar de sus notas, compartirlas con gente querida, entonarlas a los cuatro cierzos y crear ... Ver más recuerdos. Ver menos",don t hop back in anger doble ipa hay canciones convierten himnos plantas convierten cervezas en ambos casos importante disfrutar notas compartirlas gente querida entonarlas cuatro cierzos crear ver recuerdos ver menos,don t hop back in anger doble ipa hay canciones convierten himnos plantas cervezas en ambos 

## CountVectorizer

In [470]:
cvec = CountVectorizer(
    stop_words=["english", "spanish"], 
    min_df=0.01, 
    max_df=0.50, 
    ngram_range=(1,2) # change to consider bigrams
#     ngram_range=(1) # unigrams
    
)
cvec

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=0.5, max_features=None, min_df=0.01,
                ngram_range=(1, 2), preprocessor=None,
                stop_words=['english', 'spanish'], strip_accents=None,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None,
                vocabulary=None)

In [471]:
cvec.fit(df['Description_3'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=0.5, max_features=None, min_df=0.01,
                ngram_range=(1, 2), preprocessor=None,
                stop_words=['english', 'spanish'], strip_accents=None,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None,
                vocabulary=None)

In [472]:
cvec.get_feature_names()

['10',
 '100',
 'abundante',
 'acidez',
 'acompañar',
 'además',
 'aditivos',
 'afrutada',
 'afrutadas',
 'afrutado',
 'afrutados',
 'aged',
 'agradable',
 'agua',
 'ahumado',
 'ahumados',
 'al',
 'alc',
 'alcohol',
 'alcohólica',
 'alcohólico',
 'ale',
 'ale color',
 'alemán',
 'alta',
 'alta fermentación',
 'alto',
 'amantes',
 'amarga',
 'amargo',
 'amargor',
 'amargor medio',
 'amargor moderado',
 'amargura',
 'amarillo',
 'amb',
 'ambar',
 'amber',
 'amber ale',
 'american',
 'american pale',
 'americano',
 'americanos',
 'amigos',
 'anaranjado',
 'apariencia',
 'aperitivos',
 'aporta',
 'aportan',
 'aquesta',
 'aroma',
 'aroma sabor',
 'aromas',
 'aromas cítricos',
 'aromas sabores',
 'aromes',
 'aromática',
 'aromático',
 'artesana',
 'artesanal',
 'aspecto',
 'así',
 'aunque',
 'avena',
 'azúcar',
 'añadido',
 'año',
 'años',
 'baja',
 'bajo',
 'barcelona',
 'barrel',
 'barrel aged',
 'barrica',
 'barricas',
 'base',
 'bastante',
 'beber',
 'beer',
 'beige',
 'belga',
 'belgian

In [473]:
list(islice(cvec.vocabulary_.items(), 20))

[('sour', 514),
 ('fruit', 259),
 ('beer', 78),
 ('ipa', 313),
 ('nuestra', 402),
 ('mejor', 370),
 ('dry', 199),
 ('muy', 384),
 ('seca', 492),
 ('aromática', 56),
 ('amargor', 30),
 ('hops', 294),
 ('doble', 194),
 ('neipa', 392),
 ('new', 393),
 ('the', 528),
 ('amarga', 28),
 ('herbal', 288),
 ('simcoe', 511),
 ('cascade', 119)]

In [474]:
cvec_counts = cvec.transform(df['Description_3'])
print('sparse matrix shape:', cvec_counts.shape)
print('nonzero count:', cvec_counts.nnz)
print('sparsity: %.2f%%' % (100.0 * cvec_counts.nnz / (cvec_counts.shape[0] * cvec_counts.shape[1])))

sparse matrix shape: (6683, 580)
nonzero count: 115383
sparsity: 2.98%


In [475]:
# top 20 terms
occ = np.asarray(cvec_counts.sum(axis=0)).ravel().tolist()
counts_df = pd.DataFrame({'term': cvec.get_feature_names(), 'occurrences': occ})

In [477]:
counts_df.sort_values(by='occurrences', ascending=False).head(35)

,term,occurrences
21,ale,2007
142,color,1956
483,sabor,1666
50,aroma,1459
30,amargor,1397
336,lúpulo,1312
231,espuma,1289
345,malta,1255
313,ipa,1187
171,cuerpo,1176


## TfidfTransformer

In [466]:
# use tfidf transformer to calc weights
transformer = TfidfTransformer()
transformed_weights = transformer.fit_transform(cvec_counts)
transformed_weights

<6683x580 sparse matrix of type '<class 'numpy.float64'>'
	with 115383 stored elements in Compressed Sparse Row format>

In [467]:
# Compute cosine similarity with a beer of choice, that is represented in the dataset
idx = df.index.get_indexer_for((df[df['Name']=='Malandar Silk'].index))[0]

## Cosine Similarity

In [468]:
sims = cosine_similarity(transformed_weights[idx,:], transformed_weights)

# Add similarities values to the beer description dataframe, then sort by similarities descending
df['sims'] = sims.T
df.sort_values(by='sims',ascending=False)[['Name', 'Style', 'sims', 'Description_3']].head(10)

,Name,Style,sims,Description_3
687,Malandar Silk,New England IPA,1.000000,silk neipa ddh mosaic citra galaxy lactose
569,Soma Hypebeast,Imperial IPA,0.822911,hypebeast just hop it double neipa ddh cashmere citra mosaic
688,Malandar Lazy Canalla,New England IPA,0.757408,ddh neipa
970,Malandar White Trash,New England IPA,0.757408,white trash ddh neipa
652,Bonvivant Motora,New England IPA,0.757408,motora neipa ddh
969,Malandar She Is Demolitioner,New England IPA,0.757408,she is demolitioner 9 80665 ddh neipa
947,Wylie Brewery We Are Aliens,New England IPA,0.753384,we are aliens extra pale neipa amb lactosa pell taronja vainilla ddh citra mosaic azacca
294,La Grúa Bamboo,New England IPA,0.721752,bamboo ddh neipa con lúpulos idaho 7 mosaic cashmere avena
718,La Grúa Voodoo,New England IPA,0.720707,voodoo ddh neipa sabor piña coco raudales
640,Maresme Calima,New England IPA,0.670835,calima calitja hazy ipa 6 8 ddh mosaic citra hüll melon


In [469]:
df.head()

,Name,Brewery,Style,ABV,IBU,Rating,Description,Description_full,Description_2,Description_3,sims
0,BlackLab Sour Lady,NaN,Berliner Weisse,5.0,NaN,NaN,Sour Lady\nBerliner Weisse with fruit\nUna cerveza ácida con frambuesas.\n---------------\nSour beer with raspberries,Sour Lady\nBerliner Weisse with fruit\nUna cerveza ácida con frambuesas.\n---------------\nSour beer with raspberries,sour lady berliner weisse fruit una cerveza ácida frambuesas sour beer raspberries,sour lady berliner weisse fruit ácida frambuesas beer raspberries,0.000000
1,Blacklab Claudia Passion,NaN,American IPA,NaN,NaN,NaN,Claudia Passion\nFruit IPA\nNuestra mejor cerveza con Maracuyá\n--------------\nOur best IPA with passion fruit,Claudia Passion\nFruit IPA\nNuestra mejor cerveza con Maracuyá\n--------------\nOur best IPA with passion fruit,claudia passion fruit ipa nuestra mejor cerveza maracuyá our best ipa passion fruit,claudia passion fruit ipa nuestra mejor maracuyá our best,0.000000
2,BlackLab More Sparkle,NaN,Brut IPA,7.0,NaN,NaN,More Sparkle\nBrut IPA\nDry and aromatic. Low bitterness.\n-----------\nMuy seca y aromática. Poco amargor.,More Sparkle\nBrut IPA\nDry and aromatic. Low bitterness.\n-----------\nMuy seca y aromática. Poco amargor.,more sparkle brut ipa dry aromatic low bitterness muy seca aromática poco amargor,more sparkle brut ipa dry aromatic low bitterness muy seca aromática poco amargor,0.000000
3,Greyhound Brewers Give ’em Hops,NaN,Imperial IPA,8.0,NaN,NaN,Give ’em Hops\nDoble NEIPA,Give ’em Hops\nDoble NEIPA,give em hops doble neipa,give em hops doble neipa,0.343555
4,Greyhound Brewers Slave of New Hops,NaN,American IPA,6.7,50.0,3.76,Slave of New Hops\nIPA,Slave of New Hops\nIPA,slave new hops ipa,slave new hops ipa,0.000000


In [318]:
df.to_csv("Beer_dataset_v2.csv", sep='|')

# Translator

In [34]:
translator = Translator()

In [111]:
df["Description_Spanish"] = df["Description_2_clean"].map(lambda x: translator.translate(x, src="en", dest="es").text if len(x)>1 else x)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [43]:
df_test = df.iloc[0:1000]

In [54]:
df.head()

,Name,Brewery,Style,ABV,IBU,Rating,Description,Description_2,stemmed
0,Black Bitch Caravan of Bitches,La Vella Caravana,Witbier,5.2,29.0,NaN,"Caravan of Bitches\nHoppy Wheat Ale\nEstilo tradicional elaborado con malta pilsner, trigo y avena. Este estilo se caracteriza por las adiciones de cascara de naranja amarga y semillas de cilantro.","Caravan Bitches Hoppy Wheat Ale Estilo tradicional elaborado malta pilsner, trigo avena. Este estilo caracteriza adiciones cascara naranja amarga semillas cilantro.","caravan bitch hoppi wheat ale estilo tradicion elaborado malta pilsner, trigo avena. est estilo caracteriza adicion cascara naranja amarga semilla cilantro."
1,BlackLab Sour Lady,NaN,Berliner Weisse,5.0,NaN,NaN,Sour Lady\nBerliner Weisse with fruit\nUna cerveza ácida con frambuesas.\n---------------\nSour beer with raspberries,Sour Lady Berliner Weisse fruit Una cerveza ácida frambuesas. Sour beer raspberries,sour ladi berlin weiss fruit una cerveza ácida frambuesas. sour beer raspberri
2,Blacklab Claudia Passion,NaN,American IPA,NaN,NaN,NaN,Claudia Passion\nFruit IPA\nNuestra mejor cerveza con Maracuyá\n--------------\nOur best IPA with passion fruit,Claudia Passion Fruit IPA Nuestra mejor cerveza Maracuyá Our best IPA passion fruit,claudia passion fruit ipa nuestra mejor cerveza maracuyá our best ipa passion fruit
3,BlackLab More Sparkle,NaN,Brut IPA,7.0,NaN,NaN,More Sparkle\nBrut IPA\nDry and aromatic. Low bitterness.\n-----------\nMuy seca y aromática. Poco amargor.,More Sparkle Brut IPA Dry aromatic. Low bitterness. Muy seca aromática. Poco amargor.,more sparkl brut ipa dri aromatic. low bitterness. muy seca aromática. poco amargor.
4,Greyhound Brewers Give ’em Hops,NaN,Imperial IPA,8.0,NaN,NaN,Give ’em Hops\nDoble NEIPA,Give ’em Hops Doble NEIPA,give em hop dobl neipa


In [ ]:
df_test["Description_2"].map(lambda x: translator.translate(x, src="en", dest="es").text)

In [109]:
df_test = df.iloc[2000:3000]

In [324]:
df.shape

(7861, 10)

In [327]:
df.head()

,Name,Brewery,Style,ABV,IBU,Rating,Description,Description_2,Description_3,sims
0,Black Bitch Caravan of Bitches,La Vella Caravana,Witbier,5.2,29.0,NaN,"Caravan of Bitches\nHoppy Wheat Ale\nEstilo tradicional elaborado con malta pilsner, trigo y avena. Este estilo se caracteriza por las adiciones de cascara de naranja amarga y semillas de cilantro.",caravan bitches hoppy wheat ale estilo tradicional elaborado malta pilsner trigo avena este estilo caracteriza adiciones cascara naranja amarga semillas cilantro,caravan bitch hoppi wheat ale estilo tradicion elaborado malta pilsner trigo avena est caracteriza adicion cascara naranja amarga semilla cilantro,0.146398
1,BlackLab Sour Lady,NaN,Berliner Weisse,5.0,NaN,NaN,Sour Lady\nBerliner Weisse with fruit\nUna cerveza ácida con frambuesas.\n---------------\nSour beer with raspberries,sour lady berliner weisse fruit una cerveza ácida frambuesas sour beer raspberries,sour ladi berlin weiss fruit ácida frambuesa beer raspberri,0.000000
2,Blacklab Claudia Passion,NaN,American IPA,NaN,NaN,NaN,Claudia Passion\nFruit IPA\nNuestra mejor cerveza con Maracuyá\n--------------\nOur best IPA with passion fruit,claudia passion fruit ipa nuestra mejor cerveza maracuyá our best ipa passion fruit,claudia passion fruit ipa nuestra mejor maracuyá our best,0.000000
3,BlackLab More Sparkle,NaN,Brut IPA,7.0,NaN,NaN,More Sparkle\nBrut IPA\nDry and aromatic. Low bitterness.\n-----------\nMuy seca y aromática. Poco amargor.,more sparkle brut ipa dry aromatic low bitterness muy seca aromática poco amargor,more sparkl brut ipa dri aromat low bitter muy seca aromática poco amargor,0.000000
4,Greyhound Brewers Give ’em Hops,NaN,Imperial IPA,8.0,NaN,NaN,Give ’em Hops\nDoble NEIPA,give em hops doble neipa,give em hop dobl neipa,0.312885


In [343]:
translator = Translator()
df["Description_spanish"] = df["Description_2"]

for i in tqdm_notebook(range(len(df))):
    n_iter = i+1
    if n_iter % 200 == 0:
        print('Sleeping a bit...')
        time.sleep(5)   
        # reinitialize the API
        translator = Translator()            
    if len(df["Description_2"].iloc[i])>1:
        df["Description_spanish"].iloc[i] = translator.translate(df["Description_2"].iloc[i], src="en", dest="es").text
    else:
        df["Description_spanish"].iloc[i] = df["Description_2"].iloc[i]        

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [96]:
df["Description_2_clean"] = df['Description_2'].map(lambda x: re.sub(r'\W+', ' ', x))
df["stemmed_clean"] = df['stemmed'].map(lambda x: re.sub(r'\W+', ' ', x))

In [396]:
df_aux[df_aux.Description_y.isna()]

,Name,Brewery,Style,ABV,IBU,Rating,Description_x,Description_2,Description_3,sims,Description_spanish,Description_y
5082,Nox Karls Trauma,NaN,Kölsch,5.5,NaN,NaN,,,,0.0,,NaN
5084,Can Luar IPA,NaN,India Pale Ale,6.5,NaN,3.60,,,,0.0,,NaN
5093,Iberian Wheat,NaN,American Wheat or Rye Beer,6.0,NaN,3.00,,,,0.0,,NaN
5119,Capfoguer Blat,NaN,Weizen-Weissbier,4.8,NaN,3.30,,,,0.0,,NaN
5151,La Cervecita Julia`s IPA,NaN,India Pale Ale,6.9,70.0,NaN,,,,0.0,,NaN
5152,Zulogaarden Jeff Hanneman,NaN,Red Ale,6.5,60.0,NaN,,,,0.0,,NaN
5153,Zulogaarden Piños Nuevos,NaN,India Pale Ale,7.0,NaN,NaN,,,,0.0,,NaN
5154,Zulogaarden Riw-aka 47,NaN,Pale Ale,6.5,NaN,NaN,,,,0.0,,NaN
5156,Zulogaarden Wai-Ti,NaN,Pale Ale,6.5,NaN,NaN,,,,0.0,,NaN
5159,Carmen La Negra,NaN,English Brown,6.0,NaN,NaN,,,,0.0,,NaN


In [416]:
df_aux[df_aux.Description_y.str.contains('levadura', na=False)]

,Name,Brewery,Style,ABV,IBU,Rating,Description_x,Description_2,Description_3,sims,Description_spanish,Description_y
12,Gora Original Black President,NaN,Imperial Stout,10.0,NaN,NaN,"Original Black President\nImperian Stout\nGaragardoa / Cerveza / Beer\nOsagaiak: ura, garagar eta zekale maltak, olo-malutak, lupulua eta legamia.\nIngredientes: agua, malta de cebada y centeno, copos de avena, |upulo y levadura.\nIngredients: water, barley and rye malt, flacked oat, hops, yeast\nGlu ...",original black president imperian stout garagardoa cerveza beer osagaiak ura garagar eta zekale maltak olo malutak lupulua eta legamia ingredientes agua malta cebada centeno copos avena upulo levadura ingredients water barley rye malt flacked oat hops yeast glu,origin black presid imperian stout garagardoa beer osagaiak ura garagar eta zekal maltak olo malutak lupulua legamia ingredient agua malta cebada centeno copo avena upulo levadura ingredi water barley rye malt flack oat hop yeast glu,0.000000,original black president imperian stout garagardoa cerveza beer osagaiak ura garagar eta zekale maltak olo malutak lupulua eta legamia ingredientes agua malta cebada centeno copos avena upulo levadura ingredients water barley rye malt flacked oat hops yeast glu,"Original Black President\nImperian Stout\nGaragardoa / Cerveza / Beer\nOsagaiak: ura, garagar eta zekale maltak, olo-malutak, lupulua eta legamia.\nIngredientes: agua, malta de cebada y centeno, copos de avena, |upulo y levadura.\nIngredients: water, barley ... Ver más rye malt, flacked oat, hops, yeast\nGlutena dauka / Contiene gluten / Contains gluten Elaborado por “Sergio Martinez Gonzalez y otros CB” y Cerveza Byra R.G.S.\n30.013078/VI. eta “Sergio Martinez Gonzalezy otros CB”k egina.\n1..."
43,No Nation TTT,NaN,American IPA,6.5,75.0,3.52,"TTT\nWest Coast IPA\nDerma Series\nTTT Derma Series, es una cerveza de estilo West Coast con dos cepas de levadura, que han trabajado en paralelo. Una combinación perfecta de lúpulos.\n*Grabado en tu piel. Hasta el final. Lo que viviste, es lo que eres. Lo que serás, solo lo sabes tú"".",ttt west coast ipa derma series ttt derma series cerveza estilo west coast dos cepas levadura trabajado paralelo una combinación perfecta lúpulos grabado piel hasta final lo viviste eres lo serás solo sabes tú,ttt west coast ipa derma seri estilo do cepa levadura trabajado paralelo combinación perfecta lúpulo grabado piel hasta final lo vivist ere será solo sabe tú,0.000000,ttt west coast ipa derma series ttt derma series cerveza estilo west coast dos cepas levadura trabajado paralelo una combinación perfecta lúpulos grabado piel hasta final lo viviste eres lo serás solo sabes tú,"TTT\nWest Coast IPA\nDerma Series\nTTT Derma Series, es una cerveza de estilo West Coast con dos cepas de levadura, que han trabajado en paralelo. Una combinación perfecta de lúpulos.\n*Grabado en tu piel. Hasta el final. Lo que viviste, es lo que eres. ... Ver más que serás, solo lo sabes tú. Ver menos"
120,Tibidabo Brewing Jazzblat,NaN,Weizen-Weissbier,5.0,13.0,NaN,"Jazzblat\nWeissbier\nSolo la malta, el lúpulo y la levadura de fermentación alta dan unas notas agradables al plátano, a pimienta y a clavo. Sin filtrar, aportando todas las vitaminas B de la levadura, es levemente ácida y extremadamente refrescante.",jazzblat weissbier solo malta lúpulo levadura fermentación alta dan unas notas agradables plátano pimienta clavo sin filtrar aportando todas vitaminas b levadura levemente ácida extremadamente refrescante,jazzblat weissbier solo malta lúpulo levadura fermentación alta dan nota agrad plátano pimienta clavo sin filtrar aportando toda vitamina b levement ácida extremadament refrescant,0.000000,jazzblat weissbier solo malta lúpulo levadura fermentación alta dan unas notas agradables plátano pimienta clavo sin filtrar aportando todas vitaminas b levadura levemente ácida extremadamente refrescante,"Jazzblat\nWeissbier\nSolo la malta, el lúpulo y la levadura de fermentación alta dan unas not